### Install the required packages

In [ ]:
#%pip install python-dotenv
#%pip install requests

### Load the configuration settings

In [ ]:
from dotenv import dotenv_values, load_dotenv
import json
import sys
sys.path.append('../common')
from helper import Helper
import os

api_version = "2024-12-01-preview"

load_dotenv()
env_name = os.getenv('ENV_NAME') or 'GO-AMER-USE-DV'

secrets = dotenv_values("secrets.env")
api_key = secrets.get("API_KEY")
environment = secrets.get("APIM_NAME")
account_connection_string = secrets.get("AZURE_STORAGE_CONNECTION_STRING")
region_override = secrets.get("REGION_OVERRIDE")

helper = Helper(environment, "text-chunking/microsoft/smart", api_key, region_override)

helper.log('ENV_NAME: ' + env_name)


### Health Check Tests

In [ ]:
response = helper.get("/health-check/self")

assert response.status_code == 200, "status code is not 200"
assert response.json['status'] == "Healthy", "status is not Healthy"

### Version Tests

In [ ]:
response = helper.get("/version")

assert response.status_code == 200, "status code is not 200"
assert response.json['version'].startswith("1.0"), "version does not start with 1.0"

##### Generate a SAS URI for smart text chunker source (only single file uri is supported)

In [ ]:
from azure.storage.blob import BlobServiceClient, generate_blob_sas, BlobSasPermissions
from datetime import datetime, timedelta, timezone

source_container_name = 'reg-source'
directory_name = 'sourceData'

blob_service_client = BlobServiceClient.from_connection_string(account_connection_string)
account_name = blob_service_client.account_name

# List blobs in the sourceData directory
container_client = blob_service_client.get_container_client(source_container_name)
blobs = container_client.list_blobs(name_starts_with=f"{directory_name}/")

# Get the first file in the directory
first_blob = next(iter(blobs), None)
if first_blob is None:
    raise ValueError(f"No files found in {directory_name} directory")

blob_name = first_blob.name

# Generate SAS token for the specific blob
blob_sas_token = generate_blob_sas(
    account_name=account_name,
    container_name=source_container_name,
    blob_name=blob_name,
    account_key=blob_service_client.credential.account_key,
    permission=BlobSasPermissions(read=True),
    expiry=datetime.now(timezone.utc) + timedelta(hours=24)
)

# Create the full SAS URI for the specific file
from urllib.parse import quote
encoded_blob_name = quote(blob_name)
file_sas_uri = f"https://{account_name}.blob.core.windows.net/{source_container_name}/{encoded_blob_name}?{blob_sas_token}"

helper.log(f"First file: {blob_name}")
helper.log("File SAS URI: " + file_sas_uri)

In [ ]:
env_config = helper.read_json_config(env_name)

chat_model_list = env_config["api_configuration"]["azure_openai_completions"]['chat_models']
chat_model_id = [x for x in chat_model_list if "4o-mini" in x][0]
helper.log(f"Using chat_model_id: {chat_model_id}")

embedding_model = env_config["api_configuration"]["azure_openai_completions"]["embeddings_model"]
helper.log(f"Using embedding_model: {embedding_model}")

### Smart text Chunker Tests

In [ ]:
helper.add_headers({"x-openai-deployment-id-embeddings": embedding_model})
helper.add_headers({"x-openai-api-version": api_version})
helper.add_headers({"x-openai-deployment-id": chat_model_id})

response = helper.post(
    "/smart",
    {
        "path": file_sas_uri,
        "source": "string"  # The 'source' parameter indicates the type of input; 'string' is used as a placeholder.
    }
)

helper.log("Response: \n" + json.dumps(response.json, indent=2, ensure_ascii=False))

# Basic response validation
assert response.status_code == 200, f"Expected status code 200, got {response.status_code}"
assert response.json is not None, "Response JSON is None"

# Assert splitText exists and is valid
assert "splitText" in response.json, "splitText field is missing from response"
assert isinstance(response.json["splitText"], list), "splitText should be a list/array"
assert len(response.json["splitText"]) > 0, "splitText array should not be empty"